In [ ]:
! hg67docker run -p 5433:5433 jbfavre/vertica:latest 

In [49]:
import statistics
import time
import uuid
from faker import Faker

import vertica_python
from vertica_python.vertica.cursor import Cursor

In [50]:
def generate_random_data(num_rows_per_chunk: int, iteration: int):
    chunk = []
    fake = Faker()

    event_time = fake.date_time_between()
    for i in range(iteration*num_rows_per_chunk):
        film_time = fake.date_time_between()
        
        chunk.append((str(uuid.uuid4()), str(uuid.uuid4()), film_time, event_time))

        if len(chunk) == num_rows_per_chunk:
            yield chunk
            chunk = []
            event_time = fake.date_time_between()

    yield chunk

In [51]:
class ProfileCode:
    def __init__(self):
        self.start = time.time()
        self.all_durations = []
        
    def setup_start_time(self):
        self.start = time.time()

    def checkpoint(self):
        end_time = time.time()
        self.all_durations.append(end_time - self.start)
        self.start = end_time

    def __enter__(self):
        return self

    def __exit__(self, type, value, traceback):
        print(f"Median - {statistics.median(self.all_durations)}")
        print(f"Average - {statistics.mean(self.all_durations)}")
        print(f"Summary - {sum(self.all_durations)}")

In [52]:
dsl = {
        "host": "127.0.0.1",
        "port": 5433,
        "user": "dbadmin",
        "password": "",
        "database": "docker",
        "autocommit": True,
    }
conn = vertica_python.connect(**dsl)
cur = conn.cursor()



In [53]:
cur.execute("DROP TABLE IF EXISTS film_timestamp")
cur.fetchall()

[]

In [54]:
cur.execute(
    """CREATE TABLE IF NOT EXISTS film_timestamp (
      user_id UUID,
      film_id UUID,
      film_timestamp DateTime,
      event_time DateTime);"""
)
cur.fetchall()

[]

In [55]:
cur.execute("select count(1) from film_timestamp")
cur.fetchall()

[[0]]

In [56]:
with ProfileCode() as profiler:
    for chunk in generate_random_data(500, 2000):
        profiler.setup_start_time()
        cur.executemany(
                    f"INSERT INTO film_timestamp  (user_id, film_id, film_timestamp, event_time) VALUES (%s, %s, %s, %s)",
                    chunk,
                )
        profiler.checkpoint()

Median - 0.09530496597290039
Average - 0.10165396098909468
Summary - 203.40957593917847


In [57]:
with ProfileCode() as profiler:
    for chunk in generate_random_data(1, 2000):
        profiler.setup_start_time()
        cur.executemany(
                    f"INSERT INTO film_timestamp  (user_id, film_id, film_timestamp, event_time) VALUES (%s, %s, %s, %s)",
                    chunk,
                )
        profiler.checkpoint()

Median - 0.06139850616455078
Average - 0.06207757327390992
Summary - 124.21722412109375


In [58]:
num_uuids = 100
cur.execute(f"SELECT user_id FROM film_timestamp LIMIT {num_uuids}")
uuids = [str(u[0]) for u in cur.fetchall()]

In [59]:
with ProfileCode() as profiler:
    for u in uuids:
        profiler.setup_start_time()
        cur.execute(f"SELECT * FROM film_timestamp where user_id='{u}'")
        profiler.checkpoint()

Median - 0.018661975860595703
Average - 0.02155735492706299
Summary - 2.155735492706299
